# Les eit generelt endepunkt
Først litt generell kode for å kome i gang:

In [1]:
import requests
import pandas as pd
import os

parametre = {'per_page': '100'}
hode = {'Authorization': 'Bearer ' + os.environ["tokenCanvas"]}


Så definerer eg url (og dersom det er spesielle parametre kan eg redefinere dei her):

In [12]:
url = "https://hvl.instructure.com:443/api/v1/eportfolios/39986/pages"
# parametre = {"per_page": 100}

Først gjer eg eit "enkelt" kall, berre for å sjekke korleis data ser ut (men no får eg maksimalt 100 svar):

In [10]:
respons = requests.get(url, headers=hodeCanvas, params=parametreCanvas)
if 200 <= respons.status_code < 300:
    data = respons.json()
else:
    print("Feil: ", respons.status_code)

I den grad det er aktuelt kan eg så sjå på strukturen på datasettet (eller lese dokumentasjonen ...) for å bestemme meg om eg skal lage ei dataramme, eller om eg berre skal hente ut data som ei liste:

In [11]:
data[0]

{'id': 58763,
 'eportfolio_id': 39986,
 'position': 1,
 'name': 'Welcome',
 'content': [{'section_type': 'rich_text',
   'content': "<p><strong>Education</strong><br>My name is Lucas Beliën. I am in my final year of social (cultural) work at Thomas More in Geel (Belgium). <br>Social cultural work differs from the well-known careseeker - caregiver, 1 on 1, relationship. It centers around starting social learning processes, participatory practices in neighborhoods and policy, cohesion, empowerment (through group settings), etc. Its modes of practice may be different, but it is underpinned by the same social work values we are all familiar with.&nbsp;<br>The tuition fee in belgium consists of a fixed part of €131.5 and a variable part of € 5 per study point (which is usually 60/year). Alongside this there is are safety nets in place, depending the student's financial situation.<br><strong>Transport</strong><br>I used to take the bus to school very often, but then the public transport sche

Dersom eg får ein mistanke om at det er meir enn 100 svar kan eg køyre denne rutina som henter alle:

In [18]:
def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')
    return link_header_dict


dr_liste = []
hentmeir = True
while hentmeir:
    respons = requests.get(url, headers=hodeCanvas, params=parametreCanvas)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
            print(url)   # for kontrollen sin skuld
        df = pd.DataFrame(data)
        dr_liste.append(df)
# og etter at eg er ferdig å lese inn slår eg dei saman:
alledata = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)


Og så kan eg i ro og mak sjå på data:

In [19]:
alledata

,id,eportfolio_id,position,name,content,created_at,updated_at
0,58763,39986,1,Welcome,"[{'section_type': 'rich_text', 'content': '<p>...",2024-11-04T11:29:32Z,2024-11-06T18:54:50Z
1,58871,39986,1,New page,None,2024-11-06T18:22:57Z,2024-11-06T18:22:57Z
2,58870,39986,1,Assignment 2: Learning aims,"[{'section_type': 'rich_text', 'content': '<p>...",2024-11-06T18:22:48Z,2025-01-10T14:16:27Z
3,58945,39986,2,Assignment 3: Workplan,"[{'section_type': 'rich_text', 'content': '<p>...",2024-11-08T16:46:59Z,2024-11-15T20:44:04Z
4,59019,39986,3,Assignment 4: Human Rights,"[{'section_type': 'attachment', 'attachment_id...",2024-11-15T20:40:00Z,2024-11-15T20:42:00Z
5,59091,39986,4,Assignment 5: Note on social exclusion,"[{'section_type': 'attachment', 'attachment_id...",2024-11-22T12:24:15Z,2024-12-03T19:45:10Z
6,59131,39986,5,Assigment 6: Groups at risk of Social Exclusion,"[{'section_type': 'attachment', 'attachment_id...",2024-11-29T17:27:01Z,2024-12-03T19:44:59Z
7,59132,39986,6,Assignment 7: Oppressive potential,"[{'section_type': 'attachment', 'attachment_id...",2024-11-29T17:27:21Z,2024-12-03T19:45:37Z
8,59171,39986,7,Assignment 8: Note on AOP,"[{'section_type': 'attachment', 'attachment_id...",2024-12-04T23:51:16Z,2024-12-09T14:02:40Z
9,59204,39986,8,Assignment 9: Applying AOP,"[{'section_type': 'attachment', 'attachment_id...",2024-12-13T22:15:05Z,2024-12-13T22:16:14Z
